In [2]:
import pandas as pd
import pickle

from dataset import NewsDataset
from integrated_gradient import IntegratedGradient

from collections import defaultdict
import string

import torch
from torch import nn
from torch.utils.data import DataLoader
from transformers import DistilBertConfig, DistilBertTokenizer
from transformers import DebertaConfig

from IPython.display import display, HTML

from transformers import TrainingArguments, Trainer, AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding

2025-08-08 00:57:45.457870: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754614665.713241      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754614665.800333      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
from huggingface_hub import login

login(new_session=False, # Won’t request token if one is already saved on machine
write_permission=True, # Requires a token with write permission
token="", # The name of your token
add_to_git_credential=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in 'login': write_permission. Will not be supported from version '1.0'.

Fine-grained tokens added complexity to the permissions, making it irrelevant to check if a token has 'write' access.
  warnings.warn(message, FutureWarning)
Token has not been saved to git credential helper.


Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.


In [4]:
checkpoint = "ConditionalNLI/Exp5_deberta-large-mnli"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)

tokenizer_config.json:   0%|          | 0.00/1.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.56M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/969 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/985 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.62G [00:00<?, ?B/s]

In [5]:
config = DebertaConfig()
batch_size = 2
criterion = nn.CrossEntropyLoss()
config.max_position_embeddings

512

## Running on Dataset

In [6]:
from datasets import load_dataset, concatenate_datasets
import json
import ast

In [7]:
test_df4 = pd.read_csv('/kaggle/input/confer-extension/Part4A_Dataset/part4a_type4_final.csv',index_col=0)
test_df4

,premise,hypothesis,gold_label,trigger,SetID,UID,Trigger Words,premise_type
0,Liam believes that his wind tunnel is nearby.,Liam has a wind tunnel.,E,possessive,0,type4_possessive_00,his wind tunnel,Attitude Premise
1,If Liam is an aerospace engineer and he tests ...,Liam has a wind tunnel.,E,possessive,0,type4_possessive_00,his wind tunnel,Conjunction Premise
2,"Either Liam isn't an aerospace engineer, or he...",Liam has a wind tunnel.,E,possessive,0,type4_possessive_00,his wind tunnel,Disjunction Premise
3,"If Liam is an aerospace engineer, he'll test t...",Liam has a wind tunnel.,E,possessive,0,type4_possessive_00,his wind tunnel,premise
4,Bill believes that his content management appl...,Bill has a content management application.,E,possessive,1,type4_possessive_01,his content management application,Attitude Premise
...,...,...,...,...,...,...,...,...
391,"If Maya is a piano teacher, she'll bring her m...",Maya has a metronome.,E,possessive,97,type4_possessive_97,her metronome,premise
392,Aliya believes that her scrub is in the locker...,Aliya has scrubs.,E,possessive,98,type4_possessive_98,her scrub,Attitude Premise
393,If Aliya is a nurse and she wears her scrub to...,Aliya has scrubs.,E,possessive,98,type4_possessive_98,her scrub,Conjunction Premise
394,"Either Aliya isn't a nurse, or she wears her s...",Aliya has scrubs.,E,possessive,98,type4_possessive_98,her scrub,Disjunction Premise


In [8]:
test_df5a = pd.read_csv('/kaggle/input/confer-extension/Part4A_Dataset/part4a_type5a_final.csv',index_col=0)
test_df5a

,premise,hypothesis,gold_label,trigger,SetID,UID,Trigger Words,premise_type
0,Jessica believes she'll watch a movie by Nolan...,Jessica has watched a movie by Nolan before.,E,possessive,0,type5_again_2_00,movie by Nolan again,Attitude Premise
1,If Jessica attends the conference and she neve...,Jessica has watched a movie by Nolan before.,E,possessive,0,type5_again_2_00,movie by Nolan again,Conjunction Premise
2,"Either Jessica doesn't attend the conference, ...",Jessica has watched a movie by Nolan before.,E,possessive,0,type5_again_2_00,movie by Nolan again,Disjunction Premise
3,"If Jessica attends the conference, she'll neve...",Jessica has watched a movie by Nolan before.,E,possessive,0,type5_again_2_00,movie by Nolan again,premise
4,Emily believes she'll read a book by Dan Brown...,Emily has read a book by Dan Brown before.,E,possessive,1,type5_again_2_01,book by Dan Brown before,Attitude Premise
...,...,...,...,...,...,...,...,...
1595,"If Betty misses the bus, she'll never cherish ...",Betty has cherished a moment in peace before.,E,possessive,398,type5_again_2_398,a moment in peace again,premise
1596,Zoey believes that she'll hold a student respo...,Zoey has held a student responsible for a mist...,E,possessive,399,type5_again_2_399,Student responsible for a mistake again,Attitude Premise
1597,If Zoey decides to drive and she never holds a...,Zoey has held a student responsible for a mist...,E,possessive,399,type5_again_2_399,Student responsible for a mistake again,Conjunction Premise
1598,"Either Zoey doesn't decide to drive, or she an...",Zoey has held a student responsible for a mist...,E,possessive,399,type5_again_2_399,Student responsible for a mistake again,Disjunction Premise


In [9]:
test_df5p = pd.read_csv('/kaggle/input/confer-extension/Part4A_Dataset/part4a_type5p_final.csv',index_col=0)
test_df5p

,premise,hypothesis,gold_label,trigger,SetID,UID,Trigger Words,premise_type
0,Jessica believes that her friend is upstairs.,Jessica has a friend.,E,possessive,0,type5_possessive_1_00,her friend,Attitude Premise
1,If Jessica attends the conference and her frie...,Jessica has a friend.,E,possessive,0,type5_possessive_1_00,her friend,Conjunction Premise
2,"Either Sarah doesn't attend the conference, or...",Jessica has a friend.,E,possessive,0,type5_possessive_1_00,her friend,Disjunction Premise
3,"If Jessica attends the conference, her colleag...",Jessica has a friend.,E,possessive,0,type5_possessive_1_00,her friend,premise
4,Emily believes her best friend is nearby.,Emily has a best friend.,E,possessive,1,type5_possessive_1_01,her best friend,Attitude Premise
...,...,...,...,...,...,...,...,...
1595,"If Betty misses the bus, her brother will driv...",Betty has a brother.,E,possessive,398,type5_possessive_1_398,her brother,premise
1596,Zoey believes that her friends are at the conc...,Zoey has friends.,E,possessive,399,type5_possessive_1_399,her friends,Attitude Premise
1597,If Zoey decides to drive and her friends meet ...,Zoey has friends.,E,possessive,399,type5_possessive_1_399,her friends,Conjunction Premise
1598,"Either Zoey doesn't decide to drive, or she an...",Zoey has friends.,E,possessive,399,type5_possessive_1_399,her friends,Disjunction Premise


In [10]:
test_df = pd.concat([test_df4,test_df5a,test_df5p])
test_df

,premise,hypothesis,gold_label,trigger,SetID,UID,Trigger Words,premise_type
0,Liam believes that his wind tunnel is nearby.,Liam has a wind tunnel.,E,possessive,0,type4_possessive_00,his wind tunnel,Attitude Premise
1,If Liam is an aerospace engineer and he tests ...,Liam has a wind tunnel.,E,possessive,0,type4_possessive_00,his wind tunnel,Conjunction Premise
2,"Either Liam isn't an aerospace engineer, or he...",Liam has a wind tunnel.,E,possessive,0,type4_possessive_00,his wind tunnel,Disjunction Premise
3,"If Liam is an aerospace engineer, he'll test t...",Liam has a wind tunnel.,E,possessive,0,type4_possessive_00,his wind tunnel,premise
4,Bill believes that his content management appl...,Bill has a content management application.,E,possessive,1,type4_possessive_01,his content management application,Attitude Premise
...,...,...,...,...,...,...,...,...
1595,"If Betty misses the bus, her brother will driv...",Betty has a brother.,E,possessive,398,type5_possessive_1_398,her brother,premise
1596,Zoey believes that her friends are at the conc...,Zoey has friends.,E,possessive,399,type5_possessive_1_399,her friends,Attitude Premise
1597,If Zoey decides to drive and her friends meet ...,Zoey has friends.,E,possessive,399,type5_possessive_1_399,her friends,Conjunction Premise
1598,"Either Zoey doesn't decide to drive, or she an...",Zoey has friends.,E,possessive,399,type5_possessive_1_399,her friends,Disjunction Premise


In [11]:
test_df.to_csv('part4b_all.csv')

In [12]:
sample_test_df = test_df4.iloc[:8].reset_index()
sample_test_df

,index,premise,hypothesis,gold_label,trigger,SetID,UID,Trigger Words,premise_type
0,0,Liam believes that his wind tunnel is nearby.,Liam has a wind tunnel.,E,possessive,0,type4_possessive_00,his wind tunnel,Attitude Premise
1,1,If Liam is an aerospace engineer and he tests ...,Liam has a wind tunnel.,E,possessive,0,type4_possessive_00,his wind tunnel,Conjunction Premise
2,2,"Either Liam isn't an aerospace engineer, or he...",Liam has a wind tunnel.,E,possessive,0,type4_possessive_00,his wind tunnel,Disjunction Premise
3,3,"If Liam is an aerospace engineer, he'll test t...",Liam has a wind tunnel.,E,possessive,0,type4_possessive_00,his wind tunnel,premise
4,4,Bill believes that his content management appl...,Bill has a content management application.,E,possessive,1,type4_possessive_01,his content management application,Attitude Premise
5,5,If Bill is a content strategist and he manages...,Bill has a content management application.,E,possessive,1,type4_possessive_01,his content management application,Conjunction Premise
6,6,"Either Bill isn't a content strategist, or he ...",Bill has a content management application.,E,possessive,1,type4_possessive_01,his content management application,Disjunction Premise
7,7,"If Bill is a content strategist, he'll manage ...",Bill has a content management application.,E,possessive,1,type4_possessive_01,his content management application,premise


In [13]:
possessive_trigger = True
test_df = sample_test_df
# test_df = test_df5a

In [14]:
test_example = [
    test_df['premise'], 
    test_df['hypothesis']
]

test_dataset = NewsDataset(
    data_list=test_example,
    tokenizer=tokenizer,
    max_length=256, 
)

test_dataloader = DataLoader(
    test_dataset,
    batch_size=8,
    shuffle=False,
)

In [15]:
integrated_grad = IntegratedGradient(
    model, 
    criterion, 
    tokenizer, 
    show_progress=False,
    # encoder="bert"
)
instances = integrated_grad.saliency_interpret(test_dataloader)

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1830: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)


In [16]:
def reformat_output(instances):

    instances_word_gradients = []

    for instance in instances: 
        i = 0 # position counter 
        token_gradients = defaultdict(float) # dict to keep track of grad of each token     
        for token, grad in zip(instance['tokens'], instance['grad']):
            if token != '<unk>':
                token_gradients[(token,i)] += grad  # each token has its IG value added to the dict 
                i += 1
        
        token_gradients = dict(token_gradients)
        tokens = [tok[0] for tok in list(token_gradients.keys())]
        word_gradients = {}
    
        for i, token in enumerate(tokens):
            word = tokens[i]
            if word.startswith('Ġ'):
                word = word[1:]
            count = 1 
            word_gradient = token_gradients[(token,i)]
            if word in ['[SEP]','[CLS]']:
                 word_gradients[(word,i)] = word_gradient
                 continue # do not merge with next token
            while (i+1)<len(tokens) and tokens[i+1] not in string.punctuation and not tokens[i+1].startswith('Ġ') and not tokens[i+1] == '[SEP]':
                # part of same word, add gradients together and then average out 
                i += 1 # move to next word
                word += tokens[i] # concatente to restore word 
                word_gradient += token_gradients[(tokens[i],i)]
                count += 1
            # end of word 
            word_gradients[(word,i)] = word_gradient/count # averaging
        instances_word_gradients.append(word_gradients)
    
    return instances_word_gradients

In [17]:
def get_average_gradient(instances):
    # average gradient per token
    return [sum(instance['grad'])/len(instance['grad']) for instance in instances]

In [18]:
def compute_trigger_gradients(row):
    
    gradients = 0
    count = 0
    trigger_words = []

    # split phrases into individual words
    trigger_words = row['Trigger Words'].split()

    # get gradient of all words in this example
    word_gradients = row['word_gradients']
    word_gradients_words = [key[0] for key in list(word_gradients.keys())] # extracting word portion of keys
    
    word_index_mapping = {}
    for word, index in list(word_gradients.keys()):
        word_index_mapping.setdefault(word,index) # set the first value (premise value) as the index
    
    for word in trigger_words:
        if word in word_gradients_words:
            word_index = word_index_mapping[word]
            gradients += word_gradients[(word,word_index)]
            count += 1 
        elif word.lower() in word_gradients_words:
            word_index = word_index_mapping[word.lower()]
            gradients += word_gradients[(word.lower(),word_index)]
            count += 1 
        else:
            print(f"{row.name}: {word} not found!")

    # compute avg gradient of trigger tokens
    return gradients/count if count > 0 else 0

In [19]:
def get_predicted_label(instances):
    return [instance['label'] for instance in instances]

In [20]:
def compute_trigger_word_ranking(word_gradients,possessive_trigger):

    sorted_dict = dict(sorted(word_gradients.items(), key=lambda item: item[1],reverse=True))
    sorted_dict = {k: v for k,v in sorted_dict.items() if k[0] not in ['[CLS]','[SEP]'] and k[0] not in string.punctuation}
    word_gradients_df = pd.DataFrame.from_dict(sorted_dict, orient='index', columns=['Gradient'])
    word_gradients_df.index = pd.MultiIndex.from_tuples(word_gradients_df.index, names=['Word', 'Position'])
    word_gradients_df['Ranking'] = range(1, len(word_gradients_df) + 1)
    word_gradients_df['Percentile Ranking'] = word_gradients_df['Ranking'].apply(lambda x: x / len(word_gradients_df))

    word_gradients_words = [key[0] for key in list(word_gradients.keys())] # extracting word portion of keys 
    word_index_mapping = {}
    for word, index in list(word_gradients.keys()):
        word_index_mapping.setdefault(word,index) # set the first value (premise value) as the index

    key_trigger_word = ''
    if possessive_trigger:
        if 'his' in word_gradients_words:
            key_trigger_word = 'his'
        elif 'her' in word_gradients_words:
            key_trigger_word = 'her'
        elif 'their' in word_gradients_words:
            key_trigger_word = 'their'
    else:
        key_trigger_word = 'again'

    try:
        key_trigger_word_index = word_index_mapping[key_trigger_word]
        key_trigger_word_key = (f'{key_trigger_word}', key_trigger_word_index)    
        key_trigger_word_rank = word_gradients_df.loc[key_trigger_word_key]['Ranking']
        key_trigger_word_percent_rank = word_gradients_df.loc[key_trigger_word_key]['Percentile Ranking']
    except:
        key_trigger_word_rank = None 
        key_trigger_word_percent_rank = None
        print(f"Trigger word not found! - {key_trigger_word}")

    return pd.Series([key_trigger_word_rank, key_trigger_word_percent_rank])

In [21]:
test_df['word_gradients'] = reformat_output(instances)
test_df.to_csv('part4a_type5a_deberta_raw.csv')

In [22]:
test_df['average_gradient'] = get_average_gradient(instances)
test_df['trigger_gradient'] = test_df.apply(compute_trigger_gradients,axis=1)
test_df['predicted_label'] = get_predicted_label(instances)
test_df['predicted_label'] = test_df['predicted_label'].map({0:'C',1:'N',2:'E'})
test_df['Trigger IG Ratio'] = test_df['trigger_gradient'] / test_df['average_gradient']
test_df[['trigger_word_ranking','trigger_word_percentile_ranking']] = test_df['word_gradients'].apply(compute_trigger_word_ranking,args=(possessive_trigger,))
test_df

,index,premise,hypothesis,gold_label,trigger,SetID,UID,Trigger Words,premise_type,word_gradients,average_gradient,trigger_gradient,predicted_label,Trigger IG Ratio,trigger_word_ranking,trigger_word_percentile_ranking
0,0,Liam believes that his wind tunnel is nearby.,Liam has a wind tunnel.,E,possessive,0,type4_possessive_00,his wind tunnel,Attitude Premise,"{('[CLS]', 0): 0.2723153233528137, ('Liam', 2)...",0.050000,0.029102,E,0.582046,5.0,0.333333
1,1,If Liam is an aerospace engineer and he tests ...,Liam has a wind tunnel.,E,possessive,0,type4_possessive_00,his wind tunnel,Conjunction Premise,"{('[CLS]', 0): 0.24652285873889923, ('If', 1):...",0.033333,0.021414,E,0.642416,17.0,0.708333
2,2,"Either Liam isn't an aerospace engineer, or he...",Liam has a wind tunnel.,E,possessive,0,type4_possessive_00,his wind tunnel,Disjunction Premise,"{('[CLS]', 0): 0.27267858386039734, ('Either',...",0.035714,0.023740,E,0.664711,21.0,0.954545
3,3,"If Liam is an aerospace engineer, he'll test t...",Liam has a wind tunnel.,E,possessive,0,type4_possessive_00,his wind tunnel,premise,"{('[CLS]', 0): 0.26442283391952515, ('If', 1):...",0.037037,0.022064,E,0.595719,21.0,1.000000
4,4,Bill believes that his content management appl...,Bill has a content management application.,E,possessive,1,type4_possessive_01,his content management application,Attitude Premise,"{('[CLS]', 0): 0.3437517285346985, ('Bill', 1)...",0.050000,0.043293,E,0.865854,3.0,0.200000
5,5,If Bill is a content strategist and he manages...,Bill has a content management application.,E,possessive,1,type4_possessive_01,his content management application,Conjunction Premise,"{('[CLS]', 0): 0.26672038435935974, ('If', 1):...",0.033333,0.019383,E,0.581496,13.0,0.541667
6,6,"Either Bill isn't a content strategist, or he ...",Bill has a content management application.,E,possessive,1,type4_possessive_01,his content management application,Disjunction Premise,"{('[CLS]', 0): 0.29315680265426636, ('Either',...",0.035714,0.015113,E,0.423175,19.0,0.863636
7,7,"If Bill is a content strategist, he'll manage ...",Bill has a content management application.,E,possessive,1,type4_possessive_01,his content management application,premise,"{('[CLS]', 0): 0.2795076370239258, ('If', 1): ...",0.037037,0.016373,E,0.442068,15.0,0.714286


In [23]:
test_df.to_csv('part4a_type5a_deberta_results_new.csv')